<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Remaining Useful Life Forecasting using Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Predictive maintenance (PdM) of production lines is crucial for early detection of potential defects, enabling the identification and implementation of necessary maintenance activities to prevent breakdowns. A key aspect of predictive maintenance is the prediction of remaining useful life (RUL), which estimates the number of years a component in a production line can function as intended before requiring replacement.</p>

<center><img src="images/turbofan.jpg" alt="TurboFan" width=400 height=400/></center>
<p>image source: <a href="https://unsplash.com/photos/OjxpywWo9HI">unsplash.com</a></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>To achieve the goal of predict the RUL (Remaining Useful Life) of turbofan engines, We will be leveraging the power of <b>Teradata Vantage</b>, an advanced analytics platform. With Teradata Vantage, we can deploy machine learning algorithms through teradataml python library, which enable us to identify and mitigate potential machine failures before they even occur.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Teradata Vantage provides us with the necessary capabilities to analyze the vast amounts of data collected for turbofan engines , such as engine number, sensor measurement, and operational settings. By processing this data and detecting anomalies or patterns, we can take proactive measures to address potential issues, preventing costly downtimes and ensuring the longevity of the machines.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>With Teradata Vantage, we can help to client stay ahead of the curve, providing them with cutting-edge analytics capabilities to improve the reliability and efficiency of their machines.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Data Exploration</li>
    <li>Data Preparation</li>
    <li>Train-Test Split</li>
    <li>In-Database Machine Learning</li>
    <li>In-Database Model Scoring</li>
    <li>Visualize the results</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>1. Configuring the environment</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Standard Libraries
import warnings

# Data Manipulation and Analysis Libraries
import numpy as np
import pandas as pd

# Data Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# Teradata Libraries
from teradataml import *

configure.val_install_location = "val"

# Configuration
display.max_rows = 5

# Suppress Warnings
warnings.filterwarnings("ignore")

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Remaining_Useful_Life_Forecasting_PY_SQL.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>2.1 Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_RemaingUsefulLife_cloud');"        # Takes about 2 minutes
%run -i ../run_procedure.py "call get_data('DEMO_RemaingUsefulLife_local');"        # Takes about 3 minutes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style='height: 2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>3. Data Exploration</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The goal of the Predictive Maintenance (PdM) is to predict the RUL (Remaining Useful Life) of turbofan engines.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The PdM system will predict the number of remaining operational cycles before failure in the test set. i.e., the number of operational cycles after the last cycle that the engine will continue to operate. 
Also provided a vector of true Remaining Useful Life (RUL) values for the test data.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data is from 100 engines, with hundreds of rows of measurements collected from each time the engine was cycled. There are three operational settings that have a substantial effect on engine performance. Data is collected from 27 sensors. There is a row for every time the engine is cycled (starting at 1) and a column counting down the cycles until maintenance is required, labelled RUL, or Remaining Useful Life.<p/>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. 
The columns correspond to:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>unit number</li>
    <li>time, in cycles</li>
    <li>operational setting 1</li>
    <li>operational setting 2</li>
    <li>operational setting 3</li>
    <li>sensor measurement 1</li>
    <li>sensor measurement 2</li>
    <li>...</li>
    <li>sensor measurement 27</li>


</ol>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data from <a href=https://www.kaggle.com/c/predictive-maintenance>https://www.kaggle.com/c/predictive-maintenance</a> is loaded in Vantage in a table named "predictive_maintenance_rul".</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b><i>*Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</i></b></p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>3.1 Examine the predictive maintenance RUL table</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's look at the sample data in the predictive_maintenance_rul table.</p>

In [ ]:
df = DataFrame(in_schema("DEMO_RemaingUsefulLife", "predictive_maintenance_rul"))
df.sort("id")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here is a list of columns with their data types and non-null record counts.</p>

In [ ]:
df.info(null_counts=True)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have total 34 variables, out of these 28 variables are of type Float and rest 6 are Integer </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now, let's do some data exploration with <b>engine number</b> and <b>time_in_cycle</b> variables.</p>

In [ ]:
time_in_cycles_by_engineno = (
    df.groupby(["engine_no"])
    .agg({"time_in_cycles": ["count"]})
    .sort("count_time_in_cycles", False)
)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Teradata dataframe plot provides the ability to generate charts. The generated charts can be in the JPG, PNG, or SVG formats. The function provides a range of charts to visualize your data. It is used with Teradata Vantage SQL queries, allowing you to extract data from tables, and create visualizations from the result set. You can group data by one or more columns and create separate charts for each group.</p>

In [ ]:
plot = time_in_cycles_by_engineno.plot(
    x=time_in_cycles_by_engineno.engine_no,
    y=time_in_cycles_by_engineno.count_time_in_cycles,
    kind="bar",
    heading="Number of cycles completed by Engine No",
    xlabel="engine_no",
    ylabel="Number of cycles completed",
    ytick_format="999.99",
    color="blue",
    grid_linestyle="--",
    grid_linewidth=0.5,
    figsize=(900, 600),
)

# Display the plot.
plot.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that engine numbers <b>69, 92, 96, and 67</b> have more than 300 cycles completed, which is higher than other engines.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's see the average number of cycles completed per engine.</p>

In [ ]:
df.groupby(["engine_no"]).agg({"time_in_cycles": ["count"]}).agg(
    {"count_time_in_cycles": ["mean"]}
)

In [ ]:
time_in_cycles_by_engineno

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The top engine number 69 has completed 362 cycles, which is approximately 54% more cycles than the average of all the engines.</p>

<hr style='height: 2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>4. Data Preparation</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>We'll perform the following steps:</b></p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Missing Value Analysis</li>
    <li>Data distribution plot for numerical variables.</li>
    <li>Features selection using correlation</li>   
    <li>Features scaling using z-score</li>

</ul>


<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.1 Missing Value analysis</b></p>

In [ ]:
def get_missing_values_table(df):
    obj = ColumnSummary(data=df, target_columns=df.columns)
    return obj.result[["ColumnName", "NullCount", "NullPercentage"]].sort(
        ["NullPercentage", "ColumnName"], ascending=[False, True]
    )

In [ ]:
get_missing_values_table(df)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>From the above results, we see null values in the columns sensors_22 to sensors_27.</p>
<!-- There is an incomplete line here "Although these columns have 100% null values," -->

In [ ]:
# Drop columns with null
df = df.drop(
    columns=[
        "sensor_22",
        "sensor_23",
        "sensor_24",
        "sensor_25",
        "sensor_26",
        "sensor_27",
    ]
)

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.2 Distribution plots for numeric variables</b></p>

In [ ]:
def get_distribution_plot(df, cols):
    plotnumber = 1
    h, l, c = 5, len(cols), 6
    r = int(np.ceil(l / c))
    plt.figure(figsize=(20, 3 * r))
    for col in cols:
        if plotnumber <= l:
            ax = plt.subplot(r, c, plotnumber)
            plt.hist(df[[col]].get_values())
            plt.xlabel(col, fontsize=12)
        plotnumber += 1
    plt.tight_layout()
    plt.show()

In [ ]:
get_distribution_plot(df, df.columns)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>From the above results, we see some of the features like <b>sensor_1, sensor_5, sensor_10, .etc</b> values are unique. They will not contribute any values to our model development.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>So, Let's drop that columns here.</p>

In [ ]:
# Drop columns all the unique values
df = df.drop(
    columns=[
        "sensor_1",
        "sensor_5",
        "sensor_10",
        "sensor_16",
        "sensor_18",
        "sensor_19",
        "op_setting_3",
    ]
)

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.3 Checking the correlation</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we'll check the correlation of all the numeric features. Measuring correlation lets you
    determine if the value of one variable is useful in predicting the value of another.</p>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
The Sample Pearson product moment correlation coefficient is a measure of the linear association between variables. The boundary on the computed coefficient ranges from -1.00 to +1.00.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
Note that high correlation does not imply a causal relationship between the variables. The following table indicates the meaning of four extreme values for the coefficient of correlation between two variables.
</p>

<table style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <th>IF the correlation coefficient has this value</th>
    <th>THEN the association between the variables</th>
    <tr>
        <td>-1.00</td>
        <td>is perfectly linear, but inverse. <br>
        As the value for y varies, the value for x varies identically in the opposite direction.</td>
    </tr>
    <tr>
        <td>0</td>
        <td>does not exist and they are said to be uncorrelated.</td>
    </tr>
     <tr>
        <td>+1.00</td>
        <td>is perfectly linear.<br>
        As the value for y varies, the value for x varies identically in the same direction..</td>
    </tr>
</table>

In [ ]:
def get_heatmap(df):
    corr = df.to_pandas().corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    fig = px.imshow(
        corr,
        text_auto=".2f",
        width=900,
        height=1100,
        aspect="auto",
        color_continuous_scale=["lightblue", "lightyellow"],
    )
    return fig.show()

In [ ]:
get_heatmap(df)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Few observations from the correlation matrix above are:</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Time in cycle and remaining useful life (RUL) have a negative correlation with a value of -0.74.</li>
    <li>Sensor 2 and Sensor 12 have a -0.72 correlation, while Sensor 12 and Sensor 7 have a 0.81 correlation.</li>
    <li>Sensor 12 has a negative correlation with Sensor 11 and Sensor 4, with values of -0.85 and -0.82, respectively.</li>
    <li>Sensor 4 has a positive correlation with Sensor 11 and Sensor 13, with values of 0.83 and 0.75, respectively.</li>
</ul>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>While these correlations exist, we cannot train good models from the dataset. So as remedies, we can drop one column out of two columns with a higher correlation.</p>

In [ ]:
# Drop columns all the unique values
df_sel_ft = df.drop(
    columns=[
        "time_in_cycles",
        "sensor_2",
        "sensor_4",
        "sensor_7",
        "sensor_8",
        "sensor_9",
        "sensor_11",
        "sensor_12",
        "sensor_15",
        "sensor_20",
    ]
)

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.4 PairPlot for multivariate correlations</b></p>

In [ ]:
# Create a pairplot to visualize multivariate correlations
sns.pairplot(
    df_sel_ft.to_pandas()[
        ["op_setting_1", "op_setting_2", "sensor_3", "sensor_14", "sensor_13", "RUL"]
    ],
    diag_kind="auto",
    hue="RUL",
    palette="crest",
)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> From the above pairplot, we can observe the variations in each plot. The plots are in matrix format where the row name represents x axis and column name represents the y axis. The main-diagonal subplots are the univariate histograms (distributions) for each attribute.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the above pairplot, we are doing analysis of few sensors and operational settings with remaining useful life values. RUL values are divided in 5 ranges like 0-79, 80-159, etc.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>While comparing 2 values in pair along with RUL ranges colors from light green to dark blue, we observe a few things:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>When observing the 5th row with 1st column values, We can see a higher side value of RUL (dark blue) when op_setting_1 is less than 0.00 and sensor_13 values are less than 1.0.</li>
    <li>If we check the 5th-row 3rd column - when sensor_3 values are between 1570 and 1600, and sensor_13 values below 0.3 the RUL values are between 240 and 320 units.</li>
    <li>When we observe the 4th row with the 4th column we can see that data distributions in column sensor_14 are right skewed.</li>
    <li>We can see that data distributions is normal in the 1st row 1st column op_setting_1.</li>
</ol>

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.5 ZScore </b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Z-Score transforms each column value into the number of standard deviations from the mean value of the column.</p>

In [ ]:
# create ZScore obj
zs = ZScore(columns=df_sel_ft.columns[1:-1], out_columns=df_sel_ft.columns[1:-1])

# list of columns to retain in output
retain = Retain(columns="RUL")

# Process the transformation
df_transformed = valib.Transform(
    data=df_sel_ft, zscore=zs, index_columns="id", key_columns="id", retain=retain
)

<hr style='height: 2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>5. Train-Test Split</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the next step, we'll split the transformed dataset into training and testing datasets in the ratio 80:20, and we will save the datasets into Vantage.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Post spliting the dataset into train/test. Let's see number of records in train and test.</p>

In [ ]:
TrainTestSplit_out = TrainTestSplit(
    data=df_transformed.result, id_column="id", train_size=0.80, test_size=0.20, seed=7
)

df_train = TrainTestSplit_out.result[
    TrainTestSplit_out.result["TD_IsTrainRow"] == 1
].drop(["TD_IsTrainRow"], axis=1)
df_test = TrainTestSplit_out.result[
    TrainTestSplit_out.result["TD_IsTrainRow"] == 0
].drop(["TD_IsTrainRow"], axis=1)

copy_to_sql(df_train, table_name="rul_train", if_exists="replace")
copy_to_sql(df_test, table_name="rul_test", if_exists="replace")

In [ ]:
print(
    "Training Set = "
    + str(df_train.shape[0])
    + ". Testing Set = "
    + str(df_test.shape[0])
)

<hr style='height: 2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>6. In-Database Machine Learning</b>

<hr style='height: 1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>6.1 Train a XGBoost Model</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the next step, we'll use the XGBOOST function to train an xgboost model using the RUL column as the target variable for regression. XGBoost's tree-based ensemble approach, regularization techniques, handling of missing values, scalability, and feature importance capabilities make it a powerful and effective choice for modeling tabular data, often leading to superior performance compared to other machine learning algorithms.
<br>
<br>
The XGBoost function, eXtreme Gradient Boosting, implements the gradient-boosted decision tree designed for speed and performance. It has recently been dominating applied machine learning.
<br>
<br>
In gradient boosting, each iteration fits a model to the residuals (errors) of the previous iteration to correct the errors made by existing models. The predicted residual is multiplied by this learning rate and then added to the previous prediction. Models are added sequentially until no further improvements can be made. It is called gradient boosting because it uses a gradient descent algorithm to minimize the loss when adding new models.
</p>

In [ ]:
XGBoost_out = XGBoost(
    data=df_train,
    input_columns=[
        "engine_no",
        "op_setting_1",
        "op_setting_2",
        "sensor_3",
        "sensor_6",
        "sensor_13",
        "sensor_14",
        "sensor_17",
        "sensor_21",
    ],
    response_column="RUL",
    max_depth=5,
    num_boosted_trees=-1,
    model_type="regression",
    seed=123,
    lambda1=100000.0,
    shrinkage_factor=0.1,
    iter_num=10,
    column_sampling=1.0,
)

<hr style='height: 1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>6.2 XGBoost - Model Scoring</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
In the next step, we'll use the XGBoostPredict function to score the xgboost model trained in the previous step.</p>

In [ ]:
XGBoostPredict_out = XGBoost_out.predict(
    newdata=df_test,
    id_column="id",
    accumulate="RUL",
    model_type="regression",
    object_order_column=["task_index", "tree_num", "iter", "tree_order"],
).result

In [ ]:
XGBoostPredict_out

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next, we'll use the RegressionEvaluator function to evaluate the trained xgboost model on test data. This will let us know how well our model has performed on unseen data.</p>

In [ ]:
RegressionEvaluator_out = RegressionEvaluator(
    data=XGBoostPredict_out,
    observation_column="RUL",
    prediction_column="Prediction",
    freedom_degrees=[1, 28],
    independent_features_num=15,
    metrics=["MAE", "MSE", "RMSE", "R2", "FSTAT"],
)

In [ ]:
RegressionEvaluator_out.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The result table displays the evaluation metrics for XGBoost models retrieved from RegressionEvaluator. The lower the RMS error value, the better the model's performance. Here F_conclusion is <b>Reject null hypothesis</b> which means that our XGBoost model is fitting perfectly on our data. In other words, few of the predictor variables have a statistically significant relationship with the response variable, RUL.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>6.3 Train a Decision Forest Model</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Decision Forest is a powerful method used for predicting outcomes in both classification and regression problems. It's an improvement on the technique of combining (or "bagging") multiple decision trees. Normally, building a decision tree involves assessing the importance of each feature in the data to determine how to divide the information. This method takes a unique approach by only considering a random subset of features at each division point in the tree. This forces each decision tree within the "forest" to be different from one another, which ultimately improves the accuracy of the predictions. The function relies on a training dataset to develop a prediction model. Then, the TD_DecisionForestPredict function uses the model built by the TD_DecisionForest function to make predictions. It supports regression, binary, and multi-class classification tasks.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Typically, constructing a decision tree involves evaluating the value for each input feature in the data to select a split point. The function reduces the features to a random subset (that can be considered at each split point); the algorithm can force each decision tree in the forest to be very different to improve prediction accuracy. The function uses a training dataset to create a predictive model. The TD_DecisionForestPredict function uses the model created by the TD_DecisionForest function for making predictions. The function supports regression, binary, and multi-class classification.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Consider the following points:
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>All input features are numeric. Convert the categorical columns to numerical columns as preprocessing step.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>For classification, class labels (ResponseColumn values) can only be integers. A maximum of 500 classes is supported for classification.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Observations with missing values in any input column will be ignored during training. To fill in missing values, use the TD_SimpleImpute function.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>The number of trees built by the TD_DecisionForest function depends on the values of NumTrees, TreeSize, and CoverageFactor, as well as the data distribution in the cluster. The trees are built simultaneously by all the processing units (AMPs) that have a non-empty portion of the data.</li>
</p>


In [ ]:
DecisionForest_out = DecisionForest(
    data=df_train,
    input_columns=[
        "engine_no",
        "op_setting_1",
        "op_setting_2",
        "sensor_3",
        "sensor_6",
        "sensor_13",
        "sensor_14",
        "sensor_17",
        "sensor_21",
    ],
    response_column="RUL",
    max_depth=12,
    num_trees=4,
    min_node_size=1,
    mtry=3,
    mtry_seed=1,
    seed=1,
    tree_type="REGRESSION",
)



decision_df = DecisionForest_out.result

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.4 TDDecisionForestPredict</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>TDDecisionForestPredict function uses the model output by TD_DecisionForest function to analyze the input data and make predictions. This function outputs the probability that each observation is in the predicted class. Processing times are controlled by the number of trees in the model. When the number of trees is more than what can fit in memory, then the trees are cached in a local spool space.</p>

In [ ]:
DF_Predict_out = TDDecisionForestPredict(
    newdata=DataFrame("rul_test"),
    object=DecisionForest_out,
    id_column="id",
    accumulate=["engine_no", "RUL"],
)

df_result = DF_Predict_out.result
df_result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The result table displays the evaluation metrics for DecisionForest models retrieved from TD_RegressionEvaluator. The lower the RMS error value, the better the model's performance. Here F_conclusion is <b>Fail to reject null hypothesis</b> which means that our DecisionForest model is not fitting perfectly on our data. In other words, the predictor variables like sensors, op_sessions, etc. doesn't have a statistically significant relationship with the response variable, RUL</p>

In [ ]:
RegressionEvaluator_dfout = RegressionEvaluator(
    data=df_result,
    observation_column="RUL",
    prediction_column="prediction",
    freedom_degrees=[5, 48],
    independent_features_num=5,
    metrics=["RMSE", "R2", "FSTAT"],
)


RegressionEvaluator_dfout.result

<hr style='height: 2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>8. Visualize the results</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Metrics of the regression evaluator has the RMSE, R2 and the F-STAT metrics which are specified in the Metrics.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Thus here we have used 2 different models to train and predict the data. The Regression evaluator is used to evaluate and compare the models. The Teradata In-Database functions are used for training, prediction and evaluation. In this case since we have sample data the result parameters may not be accurate for these models.</p>  

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Root mean squared error (RMSE)The most common metric for evaluating linear regression model performance is called root mean squared error, or RMSE. The basic idea is to measure how bad/erroneous the model’s predictions are when compared to actual observed values. So a high RMSE is “bad” and a low RMSE is “good”.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The coefficient of determination — more commonly known as R² — allows us to measure the strength of the relationship between the response and predictor variables in the model. It’s just the square of the correlation coefficient R, so its values are in the range 0.0–1.0. Higher values of R- Squared is Good.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The metrics specified in the Metrics syntax element are displayed. For FSTAT, the following columns are displayed:</p>
<li style = 'font-size:16px;font-family:Arial'>F_score</li>
<li style = 'font-size:16px;font-family:Arial'>F_Critcialvalue</li>
<li style = 'font-size:16px;font-family:Arial'>p_value</li>
<li style = 'font-size:16px;font-family:Arial'>F_Conclusion.</li></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here we can see the comparison for MAE,MSE,RMSE and R2 for XGBoost and DecisionForest.</p> 


In [ ]:
XGB_Eval = RegressionEvaluator_out.result.to_pandas()
DF_eval = df_result.to_pandas()
result = pd.concat([DF_eval, XGB_Eval])

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above metrics compare the Decision Forest and XGBoost models. We can see that Decision Forest is performing better than XGBoost. Also, F_conclusion is Reject null hypothesis which means that our DecisionForest model is fitting perfectly on our data.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's visualize the the Decision Forest prediction result to compare actual vs. predicted values in graph.</p>

In [ ]:
# Taking first 150 records
df_result_sample = result[:150]
plt.figure(figsize=(20, 8))
plt.xlabel("ID", fontsize=14)
plt.ylabel("Remaining Useful Life", fontsize=14)
plt.plot(
    df_result_sample["id"], df_result_sample["RUL"], color="g", label="Actual Value"
)
plt.plot(
    df_result_sample["id"],
    df_result_sample["prediction"],
    color="r",
    label="Predicted Value",
)
plt.title("Actual vs Predicted using DecisionForest Classification", fontsize=18)
plt.legend()
plt.show()

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>8.1 Conclusion:</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In conclusion, the implementation of a predictive maintenance solution can greatly benefit to the client by reducing machine downtime and maintenance costs, improving production efficiency, and increasing overall productivity. Proactive scheduling of maintenance based on real-time data and analytics can help prevent costly breakdowns and emergency repairs, leading to improved machine reliability.
    <br>
    <br>
Additionally, setting limits and alarms on key parameters can enable early detection of potential failures, allowing for timely maintenance interventions. The ability to predict the type of failure can also help reduce diagnosis time, further optimizing maintenance efforts. By leveraging predictive maintenance, client can make data-driven decisions to improve their maintenance strategy, leading to tangible benefits to the company's bottom line, including increased operational efficiency, reduced costs, and improved overall performance.</p>

<hr style='height: 2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>9. Cleanup</b>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>9.1 Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ["rul_train", "rul_test"]

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'> <b>9.2 Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_RemaingUsefulLife');"        # Takes 5 seconds

In [ ]:
remove_context()

<b style = 'font-size:20px;font-family:Arial;color:#00233c'>Dataset:</b>

- `engine_no`: Unique identifier ranging from 1 to 100
- `index`: Unique row id
- `time_in_cycles`: the number of operational cycles
- `op_setting_1`: operational settings1 that have a substantial effect on engine performance.
- `op_setting_2`: operational settings2 that have a substantial effect on engine performance.
- `op_setting_3`: operational settings3 that have a substantial effect on engine performance.
- `sensor_1`: sensor measurement 1
- `sensor_2`: sensor measurement 2
- `sensor_3`: sensor measurement 3
- `sensor_4`: sensor measurement 4
- `sensor_5`: sensor measurement 5
- `sensor_6`: sensor measurement 6
- `sensor_7`: sensor measurement 7
- `sensor_8`: sensor measurement 8
- `sensor_9`: sensor measurement 9
- `sensor_10`: sensor measurement 10
- `sensor_11`: sensor measurement 11
- `sensor_12`: sensor measurement 12
- `sensor_13`: sensor measurement 13
- `sensor_14`: sensor measurement 14
- `sensor_15`: sensor measurement 15
- `sensor_16`: sensor measurement 16
- `sensor_17`: sensor measurement 17
- `sensor_18`: sensor measurement 18
- `sensor_19`: sensor measurement 19
- `sensor_20`: sensor measurement 20
- `sensor_21`: sensor measurement 21
- `sensor_22`: sensor measurement 22
- `sensor_23`: sensor measurement 23
- `sensor_24`: sensor measurement 24
- `sensor_25`: sensor measurement 25
- `sensor_26`: sensor measurement 26
- `sensor_27`: sensor measurement 27
- `RUL`: predict the number of remaining operational cycles before failure in the test set, 
i.e., the number of operational cycles after the last cycle that the engine will continue to operate. 
Also provided a vector of true Remaining Useful Life (RUL) values for the test data.


<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023,2024. All Rights Reserved
        </div>
    </div>
</footer>